<a href="https://colab.research.google.com/github/InryeolChoi/nlp_with_GPT_and_bert/blob/main/NLP_with_Bert_and_GPT%3D7_2%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install packages
< Eng >
* Install packages related to TPU.
* If you are using a TPU, remove the first character (#) in the line below and proceed. 
* If you are using a GPU, there is no need to run the line below.

< Kor >
* TPU 관련 패키지를 설치합니다.
* TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요.
* GPU를 쓴다면 아래 라인을 실행할 필요가 없습니다.

In [ ]:
# TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요.
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

* TPU 이외에 의존성 있는 패키지를 설치합니다.
* install 'ratsnlp' package

In [ ]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 582 kB 51.3 MB/s 
     |████████████████████████████████| 57 kB 5.9 MB/s 
     |████████████████████████████████| 2.8 MB 58.8 MB/s 
     |████████████████████████████████| 419 kB 64.0 MB/s 
     |████████████████████████████████| 141 kB 68.7 MB/s 
     |████████████████████████████████| 596 kB 58.0 MB/s 
     |████████████████████████████████| 101 kB 6.4 MB/s 
     |████████████████████████████████| 880 kB 36.7 MB/s 
     |████████████████████████████████| 3.3 MB 46.3 MB/s 
     |████████████████████████████████| 96 kB 8.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4cd43cf29deb521da7c58e22efa93a4d1d0a99c31633647d4585613e16912aba
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Suc

# 구글 드라이브 연동하기 (Integration with Google Drive)

* 모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 
* Connect to Google Drive to save model checkpoints and other data.

* 자신의 구글 계정에 적용됩니다.
* It will apply to your Google account

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정 (Settings)

* 모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.
* Declare the configuration information, such as model hyperparameters and save locations.

In [ ]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
    doc_stride=64,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

# 랜덤 시드 고정 (fix the seed)
* 학습 재현을 위해 랜덤 시드를 고정합니다.
* fix the seed number in order to reproduce the training

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 로거 설정 (set the logger)

* 메세지 출력 등을 위한 logger를 설정합니다.
* Set up a logger for message output and other purposes

In [ ]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=2, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)
INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=2, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=

# 말뭉치 다운로드 (download corpus)
* 실습에 사용할 말뭉치(KorQuAD-V1)를 다운로드합니다.
* download the corpus(KorQuAD-V1)

In [ ]:
nlpbook.download_downstream_dataset(args)

Downloading: 38.5MB [00:00, 106MB/s]
Downloading: 3.88MB [00:00, 99.4MB/s]                  


# 토크나이저 준비 (prepare tokenizer)
* 토큰화를 수행하는 토크나이저를 선언합니다
* declare tokenizer

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

# 학습데이터 구축 (building training data)
학습데이터를 만듭니다. (Create the training data)

In [ ]:
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = KorQuADV1Corpus()
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from train dataset file at /content/Korpora/korquad-v1
INFO:ratsnlp:Creating features from train dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 57688/57688 [06:19<00:00, 151.85it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? [SEP] 18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 하나의 교 ##향 ##곡 ##을 쓰 ##려는 뜻을 갖 ##는다 . 이 시기 바 ##그 ##너 ##는 18 ##3 ##8년 ##에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 걲 ##은 상황이 ##라 좌 ##절 ##과 실망 ##에 가득 ##했 ##으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만나는 파 ##우스 ##트 ##의 심 ##경 ##에 공감 ##했다고 한다 . 또한 파리 ##에서 아 ##브 ##네 ##크 ##의 지휘 ##로 파리 음악 ##원 관 ##현 ##악 ##단이 연 ##주 ##하는 베 ##토 [SEP]
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? [SEP] 18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 

# 테스트 데이터 구축 (Building Test Data)
* 학습 중에 평가할 테스트 데이터를 구축합니다.
* Construct test data to evaluate during training.

In [ ]:
val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from val dataset file at /content/Korpora/korquad-v1
INFO:ratsnlp:Creating features from val dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 5533/5533 [00:36<00:00, 150.91it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 198 ##9년 6월 30 ##일 평양 ##축 ##전에 대표로 파견 된 인물 ##은 ? [SEP] 198 ##9년 2월 15일 여의도 농민 폭력 시위를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처벌 ##에 ##관 ##한 ##법 ##률 ##위반 ) 으로 지명 ##수 ##배 ##되었다 . 198 ##9년 3월 12 ##일 서울 ##지 ##방 ##검찰 ##청 공안 ##부는 임종석 ##의 사전 ##구속 ##영장 ##을 발부 ##받았 ##다 . 같은 해 6월 30 ##일 평양 ##축 ##전에 임 ##수 ##경을 대표로 파견 ##하여 국가보안법 ##위반 혐의가 추가 ##되었다 . 경찰은 12월 18 ##일 ~ 20 ##일 사이 서울 경 ##희 ##대학 ##교 ##에서 임종석 ##이 성 ##명 발표 ##를 추진 ##하고 있다는 첩 ##보를 입 ##수 ##했고 , 12월 18 ##일 오전 7시 40 ##분 경 가스 ##총 ##과 [SEP]
INFO:ratsnlp:question & context: [CLS] 198 ##9년 6월 30 ##일 평양 ##축 ##전에 대표로 파견 된 인물 ##은 ? [SEP] 198 ##9년 2월 15일 여의도 농민 폭력 시위를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처벌 ##에 ##관 ##한 ##법 ##률 ##

# 모델 초기화 (Model Initialization)
* 프리트레인이 완료된 BERT 모델을 읽고, 문서 쌍 분류를 수행할 모델을 초기화합니다.
* Read a pre-trained BERT model and initialize the model for document pair classification.

In [ ]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

In [ ]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

# 학습 준비 (Preparation for Learning Process)
* Task와 Trainer를 준비합니다.
* prepare Task and Trainer

In [ ]:
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

In [ ]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# 학습 (Learning Process)
* 준비한 데이터와 모델로 학습을 시작합니다.
* Start training with the prepared data and model

* 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-doccls`)에 저장됩니다.
* The training outputs (checkpoints) will be saved in the designated location on the connected Google Drive (`/gdrive/My Drive/nlpbook/checkpoint-doccls`).

In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-qa exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/optimizer.py:381: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  f"Found unsupported keys in the optimizer configuration: {set(extra_keys)}", category=RuntimeWarning
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 108 M 
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.318   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]